# Get data and packages

In [1]:
%%capture
!pip install -U pip dask numpy fsspec>=0.3.3 tqdm pyarrow
!pip install pyarrow==0.15

In [2]:
#!aws s3 cp --recursive s3://xdss-public-datasets/demos/taxi_parquet ../datasets/taxi_parquet

In [1]:
import gc
from src.benchmarks_utils import benchmark, get_results
from src.dask_utils import *
from src.config import repetitions

use_parquet = False
name = 'dask-parquet' if use_parquet else 'dask-vaex-hdf5'
data_path = '/data/taxi_parquet/' if use_parquet else '/data/yellow_taxi_2009_2015_f32.hdf5'
results_path = f"../results/{name}_1b.csv"
benchmarks = {}
print(f"test for {repetitions} repetitions")

test for 1 repetitions


# Benchmark

In [2]:
# Load data
def read_file(df, data_path):
    if use_parquet:
        data = read_file_parquet(None, data_path)
        # we don't need to add those right?
        # data.pickup_datetime = dd.to_datetime(data.pickup_datetime)
        # data['pickup_hour'] = data.pickup_datetime.dt.hour
        # data = data.set_index('passenger_count')
    else:
        column_names = ['fare_amount', 'tip_amount', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
        data = read_file_vaex(None, data_path, chunksize=10_000_000, column_names=column_names)
    return data
data = read_file(None, data_path)
# no index is better, just leave the order as it is
# data.set_index('passenger_count')


In [3]:
print(f"size: {len(data)} with {len(data.columns)} columns")

size: 1173057927 with 8 columns


In [4]:
data

,fare_amount,tip_amount,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
npartitions=118,,,,,,,,
0,float32,float32,float32,float32,float32,float32,float32,int64
10000000,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
1170000000,...,...,...,...,...,...,...,...
1173057926,...,...,...,...,...,...,...,...


In [5]:
# %debug

In [7]:
# benchmarks['read_file']= benchmark(read_file, df=data, data_path=data_path, repetitions=repetitions)
# benchmarks['mean']= benchmark(mean, data, repetitions=repetitions)
# benchmarks['standard deviation']= benchmark(standard_deviation, data, repetitions=repetitions)
# benchmarks['mean of sum']= benchmark(mean_of_sum, data, repetitions=repetitions)
# benchmarks['mean of product']= benchmark(mean_of_product, data, repetitions=repetitions)
# benchmarks['mean of arithmetic operation']= benchmark(mean_of_complicated_arithmetic_operation, data, repetitions=repetitions)
# benchmarks['value counts']= benchmark(value_counts, data, repetitions=repetitions)
# benchmarks['groupby statistics']= benchmark(groupby_statistics, data, repetitions=repetitions)
# # benchmarks['filter']= benchmark(filter_data, data, repetitions=repetitions)
# print(f"cleaned {gc.collect()} mb")
# benchmarks['join'] = benchmark(join, data, repetitions=repetitions, other=groupby_statistics(data))
# print(f"Done benchmarks on all data")

# # filtered
# filterd = filter_data(data)
# del data

# print(f"Prepare filtered data and deleted {gc.collect()} MB")
# benchmarks['filtered length']= benchmark(length, filterd, repetitions=repetitions)
# benchmarks['filtered mean'] = benchmark(mean, filterd, repetitions=repetitions)
# benchmarks['filtered standard deviation'] = benchmark(standard_deviation, filterd, repetitions=repetitions)
benchmarks['filtered mean of sum columns'] = benchmark(mean_of_sum , filterd, repetitions=repetitions)
benchmarks['filtered mean of product'] = benchmark(mean_of_product , filterd, repetitions=repetitions)
benchmarks['filtered complicated arithmetic operation'] = benchmark(mean_of_complicated_arithmetic_operation, filterd, repetitions=repetitions)
benchmarks['filtered value counts'] = benchmark(value_counts, filterd, repetitions=repetitions)
benchmarks['filtered groupby statistics'] = benchmark(groupby_statistics, filterd, repetitions=repetitions)
benchmarks['filtered join'] = benchmark(join, filterd, repetitions=repetitions, other=groupby_statistics(filterd))
print(f"Done benchmarks on filterd data")


Done benchmarks on filterd data


In [8]:
results = get_results(benchmarks, name)
results.to_csv(results_path)
# results.head()
results

,dask-vaex-hdf5
read_file,0.013979
mean,18.139587
standard deviation,22.002922
mean of sum,19.942463
mean of product,19.711409
mean of arithmetic operation,37.855273
value counts,21.450876
groupby statistics,39.683780
join,28.438838
filtered length,44.254535


In [17]:
!aws s3 cp  ../results/dask_1b.csv s3://vaex-sagemaker-demo/benchmarks/dask_1b_results.csv 


The user-provided path ../results/dask_1b.csv does not exist.
